In [218]:
import mysql.connector.pooling as pooling;

# Task 1: Establish a connection 

In [219]:
dbconfig = {
    'user':'root',
    'password':'rootpass',
    'database':'little_lemon_db'
}

try:
    pool = pooling.MySQLConnectionPool(
        pool_name='pool_a',
        pool_size=2,
        **dbconfig
    )
    print('Connection established.')
except pooling.Error as e:
    print('Unable to establish connection.')
    print(e.msg)

Connection established.


# Task 2: Implement a stored procedure called peak_hours

In [220]:
connection1 = pool.get_connection()

In [221]:
cursor = connection1.cursor()

In [222]:
peak_hours_procedure = """
CREATE PROCEDURE IF NOT EXISTS peak_hours()
BEGIN
SELECT 
    TIME_FORMAT(booking_slot, '%H:%i:%s') booking_at, 
    COUNT(booking_id) count
FROM booking
GROUP BY TIME_FORMAT(booking_slot, '%H:%i:%s')
ORDER BY count DESC;
END
"""

In [223]:
cursor.execute(peak_hours_procedure)

In [224]:
cursor.callproc('peak_hours')

()

In [225]:
columns = next(cursor.stored_results()).column_names
results = next(cursor.stored_results())
dataset = results.fetchall()

In [226]:
for set in dataset:
    print('Booking at:', set[0])
    print('Number of Bookings:', set[1])
    print('')

Booking at: 19:00:00
Number of Bookings: 2

Booking at: 15:00:00
Number of Bookings: 1

Booking at: 17:30:00
Number of Bookings: 1

Booking at: 18:30:00
Number of Bookings: 1

Booking at: 20:00:00
Number of Bookings: 1



In [227]:
connection1.close()

# Task 3: Implement a stored procedure guest_status

In [228]:
connection1 = pool.get_connection()
cursor = connection1.cursor()

In [229]:
guest_status_procedure = """
CREATE PROCEDURE IF NOT EXISTS guest_status()
BEGIN
SELECT
    CONCAT(guest_first_name, ' ', guest_last_name) guest,
    CASE
        WHEN role IN ('Manager', 'Assistant Manager') THEN 'Ready to pay' 
        WHEN role = 'Head Chef' THEN 'Ready to serve' 
        WHEN role = 'Assistant Chef' THEN 'Preparing order'
        WHEN role = 'Head Waiter' THEN 'Order served' 
        ELSE 'Waiting'
    END order_status
FROM booking
LEFT JOIN employee USING(employee_id);
END
"""

In [230]:
cursor.execute(guest_status_procedure)

In [231]:
cursor.callproc('guest_status')

()

In [232]:
columns = next(cursor.stored_results()).column_names
results = next(cursor.stored_results())
dataset = results.fetchall()

In [233]:
for set in dataset:
    print('Guest:', set[0])
    print('Status:', set[1])
    print('')

Guest: Anna Iversen
Status: Ready to pay

Guest: Joakim Iversen
Status: Ready to pay

Guest: Vanessa McCarthy
Status: Ready to serve

Guest: Marcos Romero
Status: Preparing order

Guest: Hiroki Yamane
Status: Ready to pay

Guest: Diana Pinto
Status: Order served



In [234]:
connection1.close()